In [1]:
from PyQt5 import QtWidgets
from PyQt5 import QtCore
import win32gui
import win32ui
import win32con
import cv2
import numpy as np

ImportError: DLL load failed while importing win32ui: DLL 초기화 루틴을 실행할 수 없습니다.

In [ ]:
LABEL_LIST = ['현재색(F5)', '보색', '저장A(F2)', '저장B(F3)', 'A+B 혼합', '3x3 평균', 'HSV']

# 배경지식--

## 파이썬으로 ColorPicker를 구현하기 위해서 다른 라이브러리를 사용하지 않고 윈도우 API를 사용한다 (속도때문)
## pypiwin32 라이브러리 사용

## 윈도우 프로그래밍
### GetDesktopWindow():
####                                     Windows OS에서 최상위 윈도우를 구하는 함수
####                                     최상위 윈도우는 모든 윈도우의 부모 윈도우를 구하는 함수


### Handle:
####                                      Windows OS에서 생선되는 자원에 부여하는 고유값
####                                      핸들의 포괄적인 목적은 해당 자원을 핸들링하기 위함


### Device Context:
####                                      Windows OS에서 화면출력에 관한 모든 정보를 담고 있는 구조체
####                                      Windows에서 글자를 쓰던 그림을 그리던 DC를 이용해야 함


In [ ]:
def captureWindow(x=0, y=0, width=0, height=0):
    hwin = win32gui.GetDesktopWindow()
    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc) # 현재 보여지는 화면 --> 여기에 그리면 실제로 그려짐
    memdc = srcdc.CreateCompatibleDC() # 복사본 생성 후 작업
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)  # 압축되지 않은 원본 이미지
    memdc.SelectObject(bmp)
    memdc.BitBlt((0,0), (width, height), srcdc, (x, y), win32con.SRCCOPY)
    sigIntArray = bmp.GetBitmapBits(True)
    img = np.fromstring(sigIntArray, dtype='uint8')
    img.shape = (height, width, 4) # 4채널 RGBA
    
    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())
    
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

In [ ]:
class MyPicker(QtWidgets.QMainWindow):
    def __init__(self):
        super().__init__()
        
        self.title = '파이피커'
        self.setWindowTitle(self.title)
        self.setFixedSize(300, 500)
        self.main_widget = QtWidgets.QWidget()
        self.createWidgets()
        self.setCentralWidget(self.main_widget)
        self.show()
        
    def createWidgets(self):
        self.labelStyle = 'padding:1px; font-size:13px; font-family:맑은 고딕;'
        self.editStyle = 'padding:1px; font-size:13px; font-family:맑은 고딕; border:1px solid #000000'
        
        vbox = QtWidgets.QVBoxLayout()
        self.viewer = QtWidgets.QLabel()
        self.viewer.setBaseSize(300, 300)
        self.viewer.setText('이미지창')
        #                        가로로 가운데정렬   | 두가지 옵션 모두 적용    세로로 가운데 정렬
        self.viewer.setAlignment(QtCore.Qt.AlignCenter|QtCore.Qt.AlignVCenter)
        vbox.addWidget(self.viewer)
        
        grid = QtWidgets.QGridLayout()
        grid.setSpacing(3)
        
        for i, l in enumerate(LABEL_LIST):
            label = QtWidgets.QLabel(l)
            label.setStyleSheet(self.labelStyle)
            label.setAlignment(QtCore.Qt.AlignRight | QtCore.Qt.AlignVCenter)
            
            color = QtWidgets.QLineEdit()
            color.setStyleSheet(self.editStyle)
            color.setReadOnly(True)
            
            grid.addWidget(label, i+1, 0)
            grid.addWidget(color, i+1, 1)
        
        label = QtWidgets.QLabel('코드형식')
        label.setStyleSheet(self.labelStyle)
        label.setAlignment(QtCore.Qt.AlignRight | QtCore.Qt.AlignVCenter)
        
        self.comboBox = QtWidgets.QComboBox()
        self.comboBox.addItem('HTML')
        self.comboBox.addItem('RGB')
        self.comboBox.setStyleSheet('padding:3px;')
        grid.addWidget(label, len(LABEL_LIST) + 1, 0)
        grid.addWidget(self.comboBox, len(LABEL_LIST) + 1, 1)
        
        vbox.addLayout(grid)
        self.main_widget.setLayout(vbox)
        
# app = QtWidgets.QApplication([])
# picker = MyPicker()
# app.exec_()
c = captureWindow(0, 0, 200, 200)
cv2.imshow('S', c)
cv2.waitKey(0)
cv2.destroyAllWindows()